https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

#Importing Libraries

In [ ]:
import sys
import re
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM

In [ ]:
print(tf.__version__)

2.2.0-rc2


#Here i am downloading the dataset for text generation.
##I have downloaded the article "Leo Tolstoy's War and Peace" from adnrej-karpathy blog.
##I have used get method of requests library. Then i recieved the content in "requests.models.Response" form then i am converting it into str.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
r = requests.get("http://www.gutenberg.org/cache/epub/11/pg11.txt")

In [ ]:
# from keras.utils.data_utils import get_file
# import
# path = get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
# with io.open(path, encoding='utf-8') as f:
#     text = f.read().lower()
# print('corpus length:', len(text))
# type(text)

##Type of content i recieved.

In [ ]:
type(r)

requests.models.Response

##Converting the text recived in str form so that we can operate on it.
##I have printed first 500 text of it.

In [ ]:
raw_txt = r.text
print("The type is : {}".format(type(raw_txt)))
print(raw_txt[800:1500])

The type is : <class 'str'>
 sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversations?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so VERY remarkable in that; nor did Alice think it so
VERY much out of the way to hear the Rabbit say to itself, '


##Now i am converting the all text in lower format and using regulare expressions library for exclusion of non-asciii text.

In [ ]:
p_text = raw_txt.lower()
p_text = re.sub(r'[^\x00-\x7f]',r'', p_text)

##The type of text is

In [ ]:
print("the type of processed text is : " ,type(p_text))
type(p_text)

the type of processed text is :  <class 'str'>


str

##Printing1000 words

In [ ]:
print(p_text[900:1900])

he
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought alice 'without pictures or
conversations?'

so she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a white rabbit with pink eyes ran
close by her.

there was nothing so very remarkable in that; nor did alice think it so
very much out of the way to hear the rabbit say to itself, 'oh dear!
oh dear! i shall be late!' (when she thought it over afterwards, it
occurred to her that she ought to have wondered at this, but at the time
it all seemed quite natural); but when the rabbit actually took a watch
out of its waistcoat-pocket, and looked at it, and then hurried on,
alice started to her feet, for it flashed across her mind that she had
never before seen a rabbit with ei


#Creating a list of chars which is a set of unique chars(also sorted in ascending manner) used in the whole dataset
##Printing out the length of chars in whole dataset
##Printing the length of unique chars

In [ ]:
chars = sorted(list(set(p_text)))
print("Total length of file  : {}".format(len(p_text)))
print("Total length of chars : {}".format(len(chars)))

Total length of file  : 167515
Total length of chars : 59


##Printing out the unique set of chars in a sorted manner

In [ ]:
chars

['\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

##A neural network works with numbers, not text characters. So well  convert the characters in our input to numbers. We'll sort the list of the set of all characters that appear in our input text, then use the enumerate function to get numbers which represent the characters. We then create a dictionary that stores the keys and values, or the characters and the numbers that represent them.

##Similarly we will create a dictinary to map those numbber back to character so that when we use our model for prediction, it gives text as output.

In [ ]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

In [ ]:
ix_to_char

{0: '\n',
 1: '\r',
 2: ' ',
 3: '!',
 4: '"',
 5: '#',
 6: '$',
 7: '%',
 8: "'",
 9: '(',
 10: ')',
 11: '*',
 12: ',',
 13: '-',
 14: '.',
 15: '/',
 16: '0',
 17: '1',
 18: '2',
 19: '3',
 20: '4',
 21: '5',
 22: '6',
 23: '7',
 24: '8',
 25: '9',
 26: ':',
 27: ';',
 28: '?',
 29: '@',
 30: '[',
 31: ']',
 32: '_',
 33: 'a',
 34: 'b',
 35: 'c',
 36: 'd',
 37: 'e',
 38: 'f',
 39: 'g',
 40: 'h',
 41: 'i',
 42: 'j',
 43: 'k',
 44: 'l',
 45: 'm',
 46: 'n',
 47: 'o',
 48: 'p',
 49: 'q',
 50: 'r',
 51: 's',
 52: 't',
 53: 'u',
 54: 'v',
 55: 'w',
 56: 'x',
 57: 'y',
 58: 'z'}

##So we will create a model with character level generation
##We will create a subsequence of 80 chars which will be given as input to the model and the model will predict the 81st character.
##Similarly with the concept of LSTM, RNN, the model will also remember the value of previous output which will help in text generation or creation of a line.LSTM will easily remeber the previous chars in a sentence over a long period.

##So we have created input sequence and output sequence
##Similarly we have created x_training and y_training data which consists of integer mapped from the dictionary we created above.

In [ ]:
maxlen = 80
x_data = []
y_data = []
for i in range(0, len(p_text) - maxlen, 1):
    in_seq  = p_text[i: i + maxlen]
    out_seq = p_text[i + maxlen]
    x_data.append([char_to_ix[char] for char in in_seq])
    y_data.append([char_to_ix[out_seq]])
nb_chars = len(x_data)
print('total chars : {}'.format(nb_chars))
print('nb sentences:', int(len(x_data)/80))

total chars : 167435
nb sentences: 2092


##So we have 40726 lines each consisting of 80 chars.
##Here i am printing the first 5 words of 1st line.

In [ ]:
print(type(x_data))
x_data[1][:5]

<class 'list'>


[50, 47, 42, 37, 35]

In [ ]:
in_seq

'ebooks, and how to\r\nsubscribe to our email newsletter to hear about new ebooks.\r'

##Printing the first 5 target values of y_training data.

In [ ]:
y_data[:5]

[[34], [47], [47], [43], [2]]

##Now we'll go ahead and convert our input sequences into a processed numpy array that our network can use. We'll also need to convert the numpy array values into floats so that the sigmoid activation function our network uses can interpret them and output probabilities from 0 to 1:

In [ ]:
x = np.reshape(x_data , (nb_chars , maxlen , 1))
n_vocab = len(chars)
x = x/float(n_vocab)

In [ ]:
x.shape

(167435, 80, 1)

In [ ]:
print(type(x))
x[1][:5]

<class 'numpy.ndarray'>


array([[0.84745763],
       [0.79661017],
       [0.71186441],
       [0.62711864],
       [0.59322034]])

##Finally, we need to convert the output patterns (single characters converted to integers) into a one hot encoding. This is so that we can configure the network to predict the probability of each of the 53 different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character

In [ ]:
y = tf.keras.utils.to_categorical(y_data)

##printing out the representation of first 2 target variables.

In [ ]:
print(type(y))
y[:2]

<class 'numpy.ndarray'>


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
print("The shape of x_training data : " ,x.shape)
print("The shape of y_training data : " ,y.shape)

The shape of x_training data :  (167435, 80, 1)
The shape of y_training data :  (167435, 59)


##Creating our model with 4 LSTM layers and 1 Dense layer
###The first LSTM layer has a input shape of (80 ,1)

In [ ]:
Model = tf.keras.Sequential([
                             tf.keras.layers.LSTM(256 , input_shape = (80 , 1) , return_sequences=True),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.LSTM(256, return_sequences=True),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(59, activation= 'softmax')
])

In [ ]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 80, 256)           264192    
_________________________________________________________________
dropout (Dropout)            (None, 80, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 80, 256)           525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 59)                3

##Compiling our model

In [ ]:
Model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam')

##The network is slow to train, it takes 15 minutes to train one epoch.
##Model checkpoint is used to save the network weights at the end of the epoch 
##We will use model checkpointing to record all of the network weights to file each time an improvement in loss is observed at the end of the epoch.
##We'll save the weights and reload them when the training is finished. We'll set a checkpoint to save the weights, and then make them the callbacks for our future model.

In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
model_callbacks = [checkpoint]

In [ ]:
Model.fit(x, y , batch_size = 50, epochs = 60 , callbacks = model_callbacks)

Epoch 1/60
3349/3349 [==============================] - ETA: 0s - loss: 2.9545
Epoch 00001: loss improved from inf to 2.95446, saving model to model_weights_saved.hdf5
3349/3349 [==============================] - 67s 20ms/step - loss: 2.9545
Epoch 2/60
3349/3349 [==============================] - ETA: 0s - loss: 2.6649
Epoch 00002: loss improved from 2.95446 to 2.66489, saving model to model_weights_saved.hdf5
3349/3349 [==============================] - 67s 20ms/step - loss: 2.6649
Epoch 3/60
3349/3349 [==============================] - ETA: 0s - loss: 2.5341
Epoch 00003: loss improved from 2.66489 to 2.53405, saving model to model_weights_saved.hdf5
3349/3349 [==============================] - 67s 20ms/step - loss: 2.5341
Epoch 4/60
3349/3349 [==============================] - ETA: 0s - loss: 2.3975
Epoch 00004: loss improved from 2.53405 to 2.39754, saving model to model_weights_saved.hdf5
3349/3349 [==============================] - 67s 20ms/step - loss: 2.3975
Epoch 5/60
3349/3349

##Now we will load our model and recompileit in the same way.

In [ ]:
cd '/content/drive/My Drive'

/content/drive/My Drive


In [ ]:
Model.save('my_rnn_new_model.h5')
Model.save_weights('model_weights_saved.hdf5')

In [ ]:
Model.save_weights('my_rnn__new_model_weights.h5')

In [ ]:
filename = "model_weights_saved.hdf5"
Model.load_weights(filename)
Model.compile(loss='categorical_crossentropy', optimizer='adam')

##Now testing our model

In [ ]:
start = np.random.randint(0, len(x_data) - 2)
pattern = x_data[start]

In [ ]:
len(pattern)

80

In [ ]:
pattern = []
seed = 'there was nothing so very remarkable in that nor did alice think it so very much'
for i in seed:
    value = char_to_ix[i]
    pattern.append(value)

##Text generation from the model we trained on given training data and on 5 epochs.

In [ ]:
print(seed)
for i in range(100):
  X = np.reshape(pattern , (1, len(pattern) , 1))
  X = X/float(n_vocab)
  int_prediction = Model.predict(X , verbose = 0)
  index = np.argmax(int_prediction)
  prediction = ix_to_char[index]
  seq_in = [ix_to_char[value] for value in pattern]
  sys.stdout.write(prediction)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

there was nothing so very remarkable in that nor did alice think it so very much
 on the 
coor of the searers had aeains and she was a little shought it was 
the hatter was a little

In [ ]:
print(seed)
for i in range(1000):
  X = np.reshape(pattern , (1, len(pattern) , 1))
  X = X/float(n_vocab)
  int_prediction = Model.predict(X , verbose = 0)
  index = np.argmax(int_prediction)
  prediction = ix_to_char[index]
  seq_in = [ix_to_char[value] for value in pattern]
  sys.stdout.write(prediction)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

there was nothing so very remarkable in that nor did alice think it so very much
 shought the way of the words were tead of the
sead of the sooe of the searers took of the sea the white rabbit 
and the white rabbit shall she sat of the mouse say of the coor of the
sealers of the sea and she was a little shall had foot to the thought
the white rabbit and the white rabbit shat she was a little shought
all the way of the white rabbit shat she was a little shought ali 

sealer of the sea the white rabbit shat she was a little shought the
sears a little shought the way of the coor of the sooe of the sealen
to to the sea the white rabbit sead of the sooe of the sooe of the
sea-'

'i don't know it to the word it would not she say the sea!' said the
goython,

'i don't know it to the word it would not she sea,' said the mock turtle

sea- and she was a little shought the mooent alice had goot the satter
of the sooe of the sooe 